Preguntas
* Distinguir entre materiales y texturas
* Obtener la url desde el data
* Software compatible ¿lista?
* Tags ¿lista?

In [18]:
import random
import requests
from lxml import html
import time
import csv
import re
def updateSession(act):
    act = requests.Session()
    return act

def updateUserAgent():
    user_agent_list = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 14_4_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Mobile/15E148 Safari/604.1',
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 Edg/87.0.664.75',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18363',
    ]
    headers={"User-Agent": user_agent_list[random.randint(0, len(user_agent_list)-1)]}
    return headers
def propiedades(text, patrones):
    dicc = {}
    for key, patron in patrones.items():
        match = re.search(patron, text)
        if match:
            dicc[key] = match.group(1)
    return dicc 
def get_important(tree, xpath_code):
    dicc = {}
    propierties = tree.xpath(xpath_code)
    for element in propierties:
        text = element.text_content()
        patrones = {
            'file_size': r'File Size: (.+)',
            'id': r'Model ID: ([#]\d+)',
            'polygons': r'Polygons: (\d+|N/A)',
            'vertex': r'Vertices: (\d+|N/A)',
            'is_rigged': r'Rigged: (\w{2,3})',
            'is_animated': r'Animated: (\w{2,3})',
            'license': r'License: (.+)',
            'quality': r'Quality: (.+)',
            'aviable_formats': r'Formats: ((\.\w+)(,\s*\.\w+)*|(\.))',
        }
        dicc.update(propiedades(text, patrones))
    return dicc


def get_software(tree, xpath_code):
    dicc = {}
    properties = tree.xpath(xpath_code)
    for element in properties:
        text = element.text_content()
        texto = " ".join(text.strip().split())#Poner el texto en una solo linea
        patrones = {
            'textures': r'Textures & Materials:((\.\w+)(,\s*\.\w+)*)',
            'software': r'Software for Import:(.*?)\.\.\.',
            'materials': r'\.\w+ for (.+?) Categories',
        }
        dicc.update(propiedades(texto, patrones))

    return dicc
def get_name(tree, xpathCode):
    dicc={}
    title = tree.xpath(xpathCode)

    Name= title[0].text_content()
    '''Name'''
    patronName = r'(.+) 3D Model'
    searchName = re.search(patronName,Name)
    if searchName:
        dicc['name'] = searchName.group(1)
    return dicc
def get_dscp(tree, xpath_code):
    dicc = {}
    Dscp = tree.xpath(xpath_code)
    if len(Dscp)>0:
        dicc['description']=Dscp[0].text_content()
    else:
        dicc['description']='Not Found'
      
    return dicc
def cat_tag(respuesta):
    dicc={}
    redirection_root_lxml=html.fromstring(respuesta.text)
    categories=redirection_root_lxml.xpath("//div[@class='ads336']//span[contains(text(),'Categories:')]/parent::p/parent::div/a[not(contains(@rel,'tag'))]/text()")
    tags=redirection_root_lxml.xpath("//div[@class='ads336']//span[contains(text(),'Tags: ')]/parent::p/following-sibling::a/text()")
    dicc={'category':categories,'tags':tags}
    return dicc
class Page:
    def __init__(self,headers):
        self.url=None
        self.next_page_url=None
        self.parsed_response=None
        self.headers=headers
        self.sess = requests.Session()

    def setUrl(self,url):
        self.url=url

    def getParsedResponse(self):
        response=self.sess.get(self.url,headers=self.headers)
        self.parsed_response=html.fromstring(response.text)

    def getItemsLinks(self):
        self.getParsedResponse()
        items=self.parsed_response.xpath("//div[contains(@id,'site-')]//div[@class='itemtitle']/a/@href")
        return items
    
    def setNextPageUrl(self):
        self.getParsedResponse()
        self.next_page_url=self.parsed_response.xpath("//a[@class='nextpostslink' and @rel='next' and @aria-label='Next Page']/@href")[0]
        
    def getMaxPags(self):
        self.getParsedResponse()
        max_pags=self.parsed_response.xpath("//div[@class='nav-links']/a[last()-1]/text()")[0]
        return max_pags
    
Datos=[
       'file_size', 'id', 'name',
       'description','uri','aviable_formats',
       'textures','materials','polygons',
       'vertex','is_rigged', 'is_animated','license',
       'quality','software','category','tags'
       ]
initial_url='https://open3dmodel.com/3d-models/' #page1
headers=updateUserAgent()
page=Page(headers)
i,m=0,0
max_pags=90
with open("Modelos_3D.csv", mode="w", newline='',encoding='utf-8') as archivo: 
    escritura = csv.writer(archivo)
    #Write de header of the file
    escritura.writerow(Datos)
    while i<max_pags:
        print(f"Numero de pagina: {i+1}")
        if i==0:
            url=initial_url
        page.setUrl(url)
        page.setNextPageUrl()
        models_links=page.getItemsLinks()
        print('Max: ',len(models_links))
        for link in models_links:
                ori = page.sess
                response = ori.get(link,headers=headers)
                tree = html.fromstring(response.content)
                properties = get_important(tree, '/html/body/div[3]/div/div[5]/ul/li')
                software=get_software(tree,'/html/body/div[3]/article/div[3]/div[1]/div[3]/div[2]/div[5]')
                description=get_dscp(tree,'/html/body/div[3]/article/div[3]/div[1]/div[3]/div[1]/div[1]/p')
                name=get_name(tree,'/html/body/div[3]/article/div[3]/div[1]/div[3]/h1')   
                link={'uri': response.url}
                propleft=cat_tag(response)
                Propiedades=properties|software|description|name|link|propleft #Only works in python versions > 3.9
                Propordenadas = {key: Propiedades[key] for key in Datos} 
                m+=1
                print(f'Modelo extraido no. {m}')
                escritura.writerow(list(Propordenadas.values()))
        url=page.next_page_url
        time.sleep(random.uniform(1,3))
        i=i+1


Numero de pagina: 1
Max:  42
Modelo extraido no. 1
Modelo extraido no. 2
Modelo extraido no. 3
Modelo extraido no. 4
Modelo extraido no. 5
Modelo extraido no. 6
Modelo extraido no. 7
Modelo extraido no. 8
Modelo extraido no. 9
Modelo extraido no. 10
Modelo extraido no. 11
Modelo extraido no. 12
Modelo extraido no. 13
Modelo extraido no. 14
Modelo extraido no. 15
Modelo extraido no. 16
Modelo extraido no. 17
Modelo extraido no. 18
Modelo extraido no. 19
Modelo extraido no. 20
Modelo extraido no. 21
Modelo extraido no. 22
Modelo extraido no. 23
Modelo extraido no. 24
Modelo extraido no. 25
Modelo extraido no. 26
Modelo extraido no. 27
Modelo extraido no. 28
Modelo extraido no. 29
Modelo extraido no. 30
Modelo extraido no. 31
Modelo extraido no. 32
Modelo extraido no. 33
Modelo extraido no. 34
Modelo extraido no. 35
Modelo extraido no. 36
Modelo extraido no. 37
Modelo extraido no. 38
Modelo extraido no. 39
Modelo extraido no. 40
Modelo extraido no. 41
Modelo extraido no. 42
Numero de pagi

In [6]:
import requests 
from lxml import html

url = 'https://open3dmodel.com/3d-models/3d-character-human-body-3dsmax-model_20474.html'
xpath_code = '//*[@id="gallery"]/div[1]/div[3]/div[1]/div[2]/a'
xpath='//*[@id="page"]/div[7]/div/div/div[6]/a'

# Realizar la primera solicitud GET
response = requests.get(url)
tree = html.fromstring(response.content)
element = tree.xpath(xpath_code)
href = element[0].get('href')

# Hacer otra petición GET al enlace obtenido
response = requests.get(href)
tree = html.fromstring(response.content)
element = tree.xpath(xpath)
download_link = element[0].get('href')

# Descargar el archivo directamente desde el enlace
file_name = download_link.split('/')[-1]
with open(file_name, 'wb') as f:
    f.write(requests.get(download_link).content)

print(f"Archivo '{file_name}' descargado con éxito.")



OSError: [Errno 22] Invalid argument: '?token_id=YTdySmloTDAvMkVlenp2M1dheU1YQT09&key=NENzRzdIaktwcEswRmFsSGgrNnk1QT09'

In [11]:
import requests
from lxml import html
import time
import random

def updateSession(act):
    act = requests.Session()
    return act

def updateUserAgent():
    user_agent_list = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 14_4_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Mobile/15E148 Safari/604.1',
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 Edg/87.0.664.75',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18363',
    ]
    headers={"User-Agent": user_agent_list[random.randint(0, len(user_agent_list)-1)]}
    return headers

class Page:
    def __init__(self,headers):
        self.url=None
        self.next_page_url=None
        self.parsed_response=None
        self.headers=headers
        self.sess = requests.Session()

    def setUrl(self,url):
        self.url=url

    def getParsedResponse(self):
        response=self.sess.get(self.url,headers=self.headers)
        self.parsed_response=html.fromstring(response.text)

    def getItemsLinks(self):
        self.getParsedResponse()
        items=self.parsed_response.xpath("//div[contains(@id,'site-')]//div[@class='itemtitle']/a/@href")
        return items
    
    def setNextPageUrl(self):
        self.getParsedResponse()
        self.next_page_url=self.parsed_response.xpath("//a[@class='nextpostslink' and @rel='next' and @aria-label='Next Page']/@href")[0]
        
    def getMaxPags(self):
        self.getParsedResponse()
        max_pags=self.parsed_response.xpath("//div[@class='nav-links']/a[last()-1]/text()")[0]
        return max_pags
        
initial_url='https://open3dmodel.com/3d-models/' #page1
page=Page(headers)
i, m=0, 0
max_pags = 1
csv_filename = 'output.csv'
with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    # Write the header row
    header_row = ['path', 'file_size', 'id', 'name', 'description', 'uri', 'browse_file_formats', 'materials', 'polygons', 'vertex', 'is_rigged', 'is_animated', 'license', 'quality', 'software_for_import', 'categories', 'tags']
    csv_writer.writerow(header_row)
    while i < max_pags :
        print("new loop started")
        if i==0:
            url=initial_url
        page.setUrl(url)
        page.setNextPageUrl()
        models_links=page.getItemsLinks()
        max_pags = len(models_links)
        print('max', len(models_links))
        for link in models_links:
            ori = page.sess
            redirection_response=ori.get(link,headers=headers)
            redirection_root_lxml=html.fromstring(redirection_response.text)
            path = '' #path: Ruta al archivo descargado (en un subdirectorio data).
            file_size=redirection_root_lxml.xpath("//span[text()='File Size: ']/following-sibling::p/text()")
            id=redirection_root_lxml.xpath("//span[text()='Model ID: ']/following-sibling::p/text()")
            name=redirection_root_lxml.xpath("//h1[@class='entry-title']/text()")
            name = name[0].replace('\n', '')
            description=redirection_root_lxml.xpath("//div[@class='post-text-full']/div[@class='post-description entry-content']/p/text()")
            description=description[0].replace(',','|')
            uri=''
            browse_file_formats=redirection_root_lxml.xpath("//span[@class='notranslate']//a[starts-with(@href,'https://open3dmodel.com/')]/text()")
            materials=redirection_root_lxml.xpath("//span[text()='Textures & Materials:']/parent::p/following-sibling::p/text()")#caution!,this must be the number of materials
            materials = materials[0].replace(',', '|')
            polygons=redirection_root_lxml.xpath("//span[text()='Polygons: ']/following-sibling::p/text()") #caution!,this must be the number of polygons
            vertex=redirection_root_lxml.xpath("//span[text()='Vertices: ']/following-sibling::p/text()")
            is_rigged=redirection_root_lxml.xpath("//span[text()='Rigged: ']/following-sibling::p/text()")
            is_animated=redirection_root_lxml.xpath("//span[text()='Animated: ']/following-sibling::p/text()")
            license=redirection_root_lxml.xpath("//span[text()='License: ']/following-sibling::p/text()")
            quality=redirection_root_lxml.xpath("//span[text()='Quality: ']/following-sibling::p/text()")
            software_for_import=redirection_root_lxml.xpath("//span[contains(text(),'Textures & Materials:')]/parent::p/preceding-sibling::p[not(child::span)]/text()") #software compatible?
            software_for_import=software_for_import[0].replace(',', '|')
            categories=redirection_root_lxml.xpath("//div[@class='ads336']//span[contains(text(),'Categories:')]/parent::p/parent::div/a[not(contains(@rel,'tag'))]/text()")
            tags=redirection_root_lxml.xpath("//div[@class='ads336']//span[contains(text(),'Tags: ')]/parent::p/following-sibling::a/text()")
            '''model_info={'path':'n','file_size':file_size[0],'id':id[0],'name':name,'description':description,'uri':link,
                                'browse_file_formats':"|".join(browse_file_formats),'materials':materials,'polygons':polygons[0],'vertex':vertex[0],'is_rigged':is_rigged[0],
                                'is_animated':is_animated[0],'license':license[0],'quality':quality[0],'software_for_import':software_for_import,'categories':'|'.join(categories),'tags':'|'.join(tags)}
            print(model_info)
            models_info.append(model_info)'''
            newline = 'n' + ',' + file_size[0] + ',' + id[0].replace(' ', '').replace('#', '') + ',' + name + ',' + description + ',' + link + ',' + "|".join(browse_file_formats) + ',' + materials + ',' + polygons[0] + ',' + vertex[0] + ',' + is_rigged[0] + ',' + is_animated[0] + ',' + license[0] + ',' + quality[0] + ',' + software_for_import + ',' + "|".join(categories) + ',' + "|".join(tags)
            m += 1
            print(m)
            csv_writer.writerow(newline.split(','))
        url=page.next_page_url
        time.sleep(random.uniform(1, 3))
        i=i+1

        

new loop started
max 42
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
